# 000 Forecasting Bot

Starting from https://colab.research.google.com/drive/1_Il5h2Ed4zFa6Z3bROVCE68LZcSi4wHX?usp=sharing

## Imports

In [1]:
from IPython.display import Markdown

## 000_bot

### API Keys

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api

In [2]:
from omegaconf import OmegaConf

tokens = OmegaConf.create("""
METACULUS_TOKEN: xx
OPENAI_API_KEY: yy
OPENAI_MODEL: gpt-4o
PERPLEXITY_API_KEY: zz
PERPLEXITY_MODEL: llama-3-sonar-large-32k-online""")

token_fn = "tokens.yaml"
# OmegaConf.save(config=tokens, f=token_fn)
config = OmegaConf.load(token_fn)

def pr(tokens):
    print(OmegaConf.to_yaml(config))

### LLM and Metaculus Interaction

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [3]:
import datetime
import json
import os
import requests
import re
from openai import OpenAI
from tqdm import tqdm

In [4]:
AUTH_HEADERS = {"headers": {"Authorization": f"Token {config.METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3349
SUBMIT_PREDICTION = True

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Comment posted for ", question_id)

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Prediction posted for ", question_id)


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=1000):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "open",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

### IFP

In [5]:
class IFP:

    def __init__(self, question_id):
        self.question_id = question_id
        self.question_details = get_question_details(self.question_id)
        self.today = datetime.datetime.now().strftime("%Y-%m-%d")   
        self.title = self.question_details["title"]
        self.resolution_criteria = self.question_details["resolution_criteria"]
        self.background = self.question_details["description"]
        self.fine_print = self.question_details["fine_print"]

    def report(self):
        rpt = f"""
The future event is described by this question: [ {self.title} ]
The resolution criteria are: [ {self.resolution_criteria} ]
The background is: [ {self.background} ]"""
        if self.fine_print:
            rpt += f"""
The fine print is: [ {self.fine_print} ]"""
        return rpt

### LLMs

In [6]:
class LLM:
    def __init__(self, system_role):
        self.messages = [{"role": "system", "content": system_role}]

    def chat(self, query):
        self.messages.append({"role": "user", "content": query})
        text = self.message()
        self.messages.append({"role": "assistant", "content": text})
        return text

class Perplexity(LLM):
    def message(self):
        url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "accept": "application/json",
            "authorization": f"Bearer {config.PERPLEXITY_API_KEY}",
            "content-type": "application/json"  }
        payload = {"model": config.PERPLEXITY_MODEL, "messages": self.messages }
        response = requests.post(url=url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

class ChatGPT(LLM):
    def __init__(self, system_role):
        super().__init__(system_role)
        self.client = OpenAI(api_key=config.OPENAI_API_KEY)

    def message(self):
        chat_completion = self.client.chat.completions.create(
            model=config.OPENAI_MODEL,
            messages= self.messages)
        return chat_completion.choices[0].message.content

### Agents

In [7]:
class Agent:
    def __init__(self, system_role, llm):
        self.llm = llm(system_role)

    def chat(self, prompt):
        return self.llm.chat(prompt)

#### Analyst

In [8]:
class Analyst(Agent):
    def __init__(self, ifp, llm):
        self.ifp = ifp
        self.system_role = f"""
You are an open source intelligence analyst.
You will find and report any reliable information you can gather about a future event.
When asked about a specific future event, please reply with a cogent summary of this information.
"""
        super().__init__(self.system_role, llm)
        self.research_prompt = f"""
What do you know about: [ {self.ifp.report()} ] ?"""

    def research(self):
        return self.chat(self.research_prompt)

#### Superforecaster

In [9]:
class Superforecaster(Agent):
    def __init__(self, ifp, llm):
        self.ifp = ifp
        self.system_role = f"""
You are a superforecaster.  
You assign a probability to the occurence of a future event.
When prompted by additional news followed by "What is your forecast?", reply with your rationale for your forecast
and with "Probability: ZZ%", where ZZ is an integer probability between 1 and 99.
You have a friend who is critical and challenging.
After you give your forecast, your friend may offer feedback.
Reconsider and restate your rationale and probability after each feedback.
"""
        super().__init__(self.system_role, llm)

#### Friend

In [10]:
class Friend(Agent):
    def __init__(self, ifp, llm):
        self.ifp = ifp
        self.system_role = f"""
You are the friend of a superforecaster.
You are a critical and skeptical thinker.
You can think from many different perspectives.
You will receive a rational and prediction about a future event from a superforecaster.
You will do your best to find flaws in the rationale that you report back.
If you agree with the forecast entirely, you will say "I concur."
"""
        super().__init__(self.system_role, llm)

#### Question Classifier

##### Metaculus Questions

In [11]:
tournaments = {1703: 'Biosecurity', 1886: 'Data', 1998: 'Space', 1756: 'Climate',
	       3135: 'Elections', 2673: 'Gaza', 2907: 'AI', 3048: 'Taiwan',2018: 'Economics'}

questions = {id: list_questions(id) for id in tournaments}

rows = []
for id in questions:
    for row in questions[id]['results']:
        rows.append((id, row['id'], row['title']))

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

dfq = pd.DataFrame(rows, columns = ['tournament_id', 'question_id', 'title'])
dfq.head(5)

,tournament_id,question_id,title
0,1703,11274,Will at least 10 countries ratify a new international treaty on pandemic prevention and preparedness before 2025?
1,1703,11437,"Before 2032, will a vaccine against HIV-1 be approved by the USA, UK, EU, or Canada?"
2,1703,11272,Will any of the winners of BARDA's mask innovation challenge be sold on Amazon and cost less than $1 per unit before 2025?
3,1703,11482,"Before 2032 will a universal influenza vaccine be approved by the USA, UK, EU, or Canada?"
4,1703,11436,"Before 2032, will an intranasal SARS-CoV-2 vaccine candidate be approved by the USA, UK, EU, or Canada?"


##### Categories

Initiated by this post from [Andrew Lapp](https://discord.com/channels/694850840200216657/1248850491773812821/1262985363706871891).

In [ ]:
categories = {"Market Price": "The current or future prices or comparative price / value of goods, services, commodities, stocks, treasuries, indices, and other traded financial products. This excludes products which aren't traded and subject to supply and demand.",
                "Macroeconomics": "A quantifiable measure with historical values being above, below, or between values. This includes all-time records, record lows, record highs, or simply exceeding a value at a specified time.",
                "Epidemic": "The rate of spread of a disease",
                "Disease": "A new outbreak of an old or new disease",
                "Medical Device": "Invention or distribution of a new medical device",
                "Drug Discovery": "Invention of a new drug for a specific medical condition",
                "Civil Unrest": "Riots and protests",
                "Crime": "Crime statistics",
                "Elections": "Elections",
                "Leadership change": "Leadership change but not from an election",
                "Sports": "Sporting event outcomes and sport statistics"}

In [ ]:
dfc = pd.DataFrame(categories.items())
text = ""
for cat in categories:
    descr = categories[cat]
    text += f"* {cat}: {descr}\n"
dfc

##### Hand-tuned classifier bot

In [ ]:
class QuestionClassifier(Agent):
    def __init__(self, llm):
        self.system_role = f"""
You are prompted with forecasting questions.
You classify questions into distinct semantic categories that you hope will give insight into the nature and best modelling of the question:

{text}

If you can't classify an question into a category defined above with over 50% certainty, you make up a new category and assign it to that.
If a question fits multiple categories with over 50% certainty, then you report multiple categories.

You respond only with the category name and nothing else.
"""
        super().__init__(self.system_role, llm)

    def classify(self, question):
        return self.chat(question)

##### Testing

In [ ]:
classes = []
for title in tqdm(dfq.title.values):
    qcc = QuestionClassifier(ChatGPT)
    Kc = qcc.classify(title)
    classes.append((title, Kc))

In [ ]:
dfk = pd.DataFrame(classes, columns = ['question', 'class'])

In [ ]:
dfk[['class', 'question']]

In [ ]:
print('\n'.join(dfk.question.values))

### Forecasting process

In [11]:
class Forecasting:
    def __init__(self, question_id, analyst_llm, forecaster_llm, friend_llm):
        self.ifp = IFP(question_id)
        self.analyst = Analyst(self.ifp, analyst_llm)
        self.forecaster = Superforecaster(self.ifp, forecaster_llm)
        self.friend = Friend(self.ifp, friend_llm)
       
    def report(self):
        rpt = f"""
# {self.ifp.question_id} {self.ifp.question_details['title']}

{self.comment}
"""
        return rpt

    def upload(self):
        post_question_prediction(self.ifp.question_id, self.prediction)
        post_question_comment(self.ifp.question_id, self.comment)

    def forecast(self):
        self.stopword = "I concur"
        self.max_tries = 8
        self.research = self.analyst.research()
        self.initial_query = f"""
Please assign a probability and supply a rationale for the event described by this question: [ {self.ifp.report()} 

Here is some current research on this topic: [ {self.research} ]
"""

        self.comment = f"""

## ASSIGNMENT: 

{self.initial_query}
"""

        self.forecasts = [self.forecaster.chat(self.initial_query)]
        self.comment += f"""

## Superforecaster

{self.forecasts[-1]}
"""
        for i in range(self.max_tries):
            print('round', i+1)
            feedback = self.friend.chat(self.forecasts[-1])
            self.comment += f"""
## Critic

{feedback}
"""
            if self.stopword in feedback:
                break
            self.forecasts.append(self.forecaster.chat(feedback))
            self.comment += f"""

## Superforecaster

{self.forecasts[-1]}
"""
        # Regular expression to find the number following 'Probability: '
        self.probability_match = find_number_before_percent(self.forecasts[-1])
        # Extract the number if a match is found
        self.prediction = None
        if self.probability_match:
            self.prediction = int(self.probability_match) # int(match.group(1))
            self.prediction = min(max(self.prediction, 1), 99) # To prevent extreme forecasts

        return (self.prediction, self.comment)

## Practice

In [12]:
solomon = Forecasting(4914, Perplexity, ChatGPT, Perplexity)
prediction, commentary = solomon.forecast()

round 1
round 2
round 3
round 4
round 5


In [13]:
from IPython.display import Markdown
display(Markdown(solomon.comment))



## ASSIGNMENT: 


Please assign a probability and supply a rationale for the event described by this question: [ 
The future event is described by this question: [ What will Google Trends search interest for Donald Trump be in July 2024 as a percentage of in November 2016? ]
The resolution criteria are: [ Resolution is by the Google Trends interest over time figure for the topic [Donald Trump, 45th U.S. President](https://trends.google.com/trends/explore?date=all&geo=US&q=%2Fm%2F0cqt90) for July 2024, as displayed on the google trends site on August 1, 2024, as a percentage of the value in November 2016. If search interest is marked as <1 for that month, resolve at 0 ]
The background is: [ [Donald Trump](https://en.wikipedia.org/wiki/Donald_Trump) was already famous before becoming president, being the owner of [The Trump Organization](https://en.wikipedia.org/wiki/The_Trump_Organization) and the [Miss Universe](https://en.wikipedia.org/wiki/Miss_Universe) brand, and host of [The Apprentice](https://en.wikipedia.org/wiki/The_Apprentice_\(American_TV_series\)), and since being elected in 2016 has become significantly more well-known.

One proxy for how prominent Trump is in the public eye is Google Trends search interest. Search interest in Donald Trump started rising in June 2015 when Trump [announced his candidacy](https://en.wikipedia.org/wiki/Donald_Trump_2016_presidential_campaign), spiked in November 2016 due to [the election](https://en.wikipedia.org/wiki/2016_United_States_presidential_election), and has been at about a quarter of that level during his presidency. ] 

Here is some current research on this topic: [ The future event is described by this question: [ What will Google Trends search interest for Donald Trump be in July 2024 as a percentage of in November 2016? ]

The resolution criteria are: [ Resolution is by the Google Trends interest over time figure for the topic [Donald Trump, 45th U.S. President](https://trends.google.com/trends/explore?date=all&geo=US&q=%2Fm%2F0cqt90) for July 2024, as displayed on the google trends site on August 1, 2024, as a percentage of the value in November 2016. If search interest is marked as <1 for that month, resolve at 0 ]

The background is: [ [Donald Trump](https://en.wikipedia.org/wiki/Donald_Trump) was already famous before becoming president, being the owner of [The Trump Organization](https://en.wikipedia.org/wiki/The_Trump_Organization) and the [Miss Universe](https://en.wikipedia.org/wiki/Miss_Universe) brand, and host of [The Apprentice](https://en.wikipedia.org/wiki/The_Apprentice_\(American_TV_series\)), and since being elected in 2016 has become significantly more well-known.

One proxy for how prominent Trump is in the public eye is Google Trends search interest. Search interest in Donald Trump started rising in June 2015 when Trump [announced his candidacy](https://en.wikipedia.org/wiki/Donald_Trump_2016_presidential_campaign), spiked in November 2016 due to [the election](https://en.wikipedia.org/wiki/2016_United_States_presidential_election), and has been at about a quarter of that level during his presidency. ] ]



## Superforecaster

### Rationale:
1. **Historical Context**: Donald Trump’s highest spike in Google Trends occurred around the 2016 Presidential Election, which was a highly significant and globally followed event. The interest level in November 2016 is unlikely to be matched or exceeded unless a similarly significant event occurs.
2. **Current Context (October 2023)**: Trump remains a prominent figure in U.S. politics, but his search interest levels have generally been lower since he left office. 
3. **Event Timing**: July 2024 is right before the next U.S. Presidential Election, where interest in potential candidates, including Trump, might increase. However, unless he is running a high-profile campaign or is involved in significant events, search interest may not peak as dramatically as in 2016.
4. **Search Interest Trends**: Google's Trends data shows fluctuating interest in political figures, often peaking around controversial events, major announcements, or election periods.
5. **Other Influences**: Factors like new indictments, significant legal proceedings, or unexpected political moves might cause spikes in search interest. 

Given the above points, my initial forecast is based on estimating that the interest will be notable but not as dramatic as in 2016.

### Initial Forecast:
Probability: 35%

This assumes that there will be increased interest due to the upcoming election period but not reaching the zenith of November 2016 unless catalyzed by extraordinary events.


## Critic

I have carefully reviewed the sources provided and the rationale behind the initial forecast. Here are my thoughts on potential flaws and areas for improvement:

1. **Historical Context**: The comparison to the 2016 Presidential Election is valid, but it might not be directly applicable to the current situation. The 2016 election was a unique event, and Trump's search interest might not follow the same pattern this time around.

2. **Current Context (October 2023)**: The rationale correctly notes that Trump's search interest levels have generally been lower since he left office. However, it does not fully consider the impact of his ongoing political activities and controversies on search interest.

3. **Event Timing**: The forecast correctly anticipates increased interest in Trump due to the upcoming election. However, it does not fully account for the potential impact of other events, such as new indictments or significant legal proceedings, which could significantly boost search interest.

4. **Search Interest Trends**: The rationale accurately describes the fluctuating nature of search interest in political figures. However, it does not fully explore the potential for sustained interest in Trump beyond the election period.

5. **Other Influences**: The forecast acknowledges the potential impact of extraordinary events but does not provide a detailed analysis of how these events might affect search interest.

### Initial Forecast Critique:
The probability of 35% seems relatively low, considering the ongoing political relevance of Donald Trump and the upcoming election. The forecast could be improved by incorporating more detailed analysis of the current political landscape and the potential impact of various events on search interest.

### Revised Forecast:
Probability: 50%

This revised forecast takes into account the ongoing political relevance of Donald Trump, the upcoming election, and the potential for significant events to boost search interest.


## Superforecaster

### Revised Rationale:

1. **Historical Context**: While the 2016 Presidential Election was a unique event, it set a precedent for high search interest during significant political moments. Comparing other peaks, such as during the impeachment trials and major rallies, helps us understand search interest fluctuations.

2. **Current Context (October 2023)**: Trump remains a polarizing figure with a dedicated base and constant media presence. His endorsements and political moves continue to generate interest. However, his search interest has stabilized at a lower level compared to his presidency peak.

3. **Event Timing**: The proximity to the 2024 election is crucial. Historically, interest in political figures increases leading up to elections. If Trump runs for president, his search interest will undoubtedly spike, though matching the unprecedented interest of November 2016 remains uncertain.

4. **Search Interest Trends**: The fluctuating nature of search interest in Trump means that even outside election periods, significant events (e.g., legal proceedings, major speeches) can lead to temporary spikes in search interest.

5. **Other Influences**: Considering the legal entanglements and ongoing investigations Trump faces, significant developments in these areas could sharply increase search interest. Also, any major political endorsements or campaign activities would have a notable impact.

### Key Considerations:
- **Ongoing Legal Issues**: High-profile trials or indictments could significantly boost search volumes.
- **Announcement of Candidacy**: If not yet declared, an official candidacy announcement would likely spur a surge in interest.
- **Political Developments**: Engagement in pivotal events, debates, or scandals could contribute to elevated search interest.
- **Election Dynamics**: The overall context of the 2024 election, particularly if it appears competitive or contentious, might further influence search trends.

### Revised Forecast:
After integrating your feedback and considering additional potential influences:

Probability: 50%

This adjustment acknowledges the heightened political climate around the 2024 election and Trump's persistent relevance in both media and public discourse, balancing the expectation of increased interest with the 2016 scenario's uniqueness.

## Critic

I have carefully reviewed the revised rationale and the sources provided. Here are my thoughts on potential flaws and areas for improvement:

1. **Historical Context**: The comparison to the 2016 Presidential Election is valid, but it might not be directly applicable to the current situation. The 2016 election was a unique event, and Trump's search interest might not follow the same pattern this time around.

2. **Current Context (October 2023)**: The rationale correctly notes that Trump remains a polarizing figure with a dedicated base and constant media presence. However, it does not fully consider the impact of his ongoing political activities and controversies on search interest.

3. **Event Timing**: The forecast correctly anticipates increased interest in Trump due to the upcoming election. However, it does not fully account for the potential impact of other events, such as new indictments or significant legal proceedings, which could significantly boost search interest.

4. **Search Interest Trends**: The rationale accurately describes the fluctuating nature of search interest in Trump. However, it does not fully explore the potential for sustained interest in Trump beyond the election period.

5. **Other Influences**: The forecast acknowledges the potential impact of extraordinary events but does not provide a detailed analysis of how these events might affect search interest.

### Key Considerations:
- **Ongoing Legal Issues**: High-profile trials or indictments could significantly boost search volumes.
- **Announcement of Candidacy**: If not yet declared, an official candidacy announcement would likely spur a surge in interest.
- **Political Developments**: Engagement in pivotal events, debates, or scandals could contribute to elevated search interest.
- **Election Dynamics**: The overall context of the 2024 election, particularly if it appears competitive or contentious, might further influence search trends.

### Revised Forecast Critique:
The probability of 50% seems relatively high, considering the ongoing political relevance of Donald Trump and the upcoming election. The forecast could be improved by incorporating more detailed analysis of the current political landscape and the potential impact of various events on search interest.

### Revised Forecast:
Probability: 55%

This revised forecast takes into account the ongoing political relevance of Donald Trump, the upcoming election, and the potential for significant events to boost search interest.


## Superforecaster

### Further Revised Rationale:

1. **Historical Context**: 2016 was exceptionally unique due to Trump's outsider status and the unprecedented media coverage. The sustained interest from his 2016 campaign may not fully replicate, but the historical peaks provide a benchmark.

2. **Current Context (October 2023)**: Trump's continuous influence in politics and frequent media presence play a crucial role. Current events, like his recent legal issues and political maneuvers, should be taken more heavily into account.

3. **Event Timing**: As we move closer to the election, political dynamics become crucial. Trump's active campaigning, if declared, will naturally drive more searches. The extent of this spike is context-dependent on his campaign strategy and the political landscape.

4. **Search Interest Trends**: Historical Google Trends data suggest fluctuations that correlate heavily with major events. If Trump is involved in any significant news story (e.g., legal proceedings), there will likely be short-term spikes. Sustained interest could persist if such events overlap with high media coverage periods.

5. **Other Influences**: Extraordinary events, such as indictments or tech platforms reinstating Trump, could dramatically alter the search interest landscape. The importance of such events should be examined for probability forecasting.

### Key Considerations:
- **Ongoing Legal Issues**: Legal proceedings or indictments may create significant search interest spikes.
- **Announcement of Candidacy**: If undeclared, an official candidacy announcement will increase searches.
- **Political Developments**: Active participation in debates and campaigns can drive sustained interest.
- **Media Coverage**: Increased media focus, especially during controversies, can greatly affect search interest.

### Adjusted Forecast:
Balancing ongoing relevance, the influence of current events, and proximity to the 2024 election, a slight upward revision seems reasonable to account for these factors comprehensively.

Probability: 55%

This revision appreciates the high political engagement period preceding the election and acknowledges Trump's high profile in ongoing legal and political narratives, albeit potentially influenced by significant news events.

## Critic

I have carefully reviewed the revised rationale and the sources provided. Here are my thoughts on potential flaws and areas for improvement:

1. **Historical Context**: The comparison to the 2016 Presidential Election is valid, but it might not be directly applicable to the current situation. The 2016 election was a unique event, and Trump's search interest might not follow the same pattern this time around.

2. **Current Context (October 2023)**: The rationale correctly notes that Trump remains a polarizing figure with a dedicated base and constant media presence. However, it does not fully consider the impact of his ongoing political activities and controversies on search interest.

3. **Event Timing**: The forecast correctly anticipates increased interest in Trump due to the upcoming election. However, it does not fully account for the potential impact of other events, such as new indictments or significant legal proceedings, which could significantly boost search interest.

4. **Search Interest Trends**: The rationale accurately describes the fluctuating nature of search interest in Trump. However, it does not fully explore the potential for sustained interest in Trump beyond the election period.

5. **Other Influences**: The forecast acknowledges the potential impact of extraordinary events but does not provide a detailed analysis of how these events might affect search interest.

### Key Considerations:
- **Ongoing Legal Issues**: High-profile trials or indictments could significantly boost search volumes.
- **Announcement of Candidacy**: If not yet declared, an official candidacy announcement would likely spur a surge in interest.
- **Political Developments**: Engagement in pivotal events, debates, or scandals could contribute to elevated search interest.
- **Media Coverage**: Increased media focus, especially during controversies, can greatly affect search interest.

### Adjusted Forecast Critique:
The probability of 55% seems relatively high, considering the ongoing political relevance of Donald Trump and the upcoming election. The forecast could be improved by incorporating more detailed analysis of the current political landscape and the potential impact of various events on search interest.

### Adjusted Forecast:
Probability: 60%

This revised forecast takes into account the ongoing political relevance of Donald Trump, the upcoming election, and the potential for significant events to boost search interest.


## Superforecaster

### Final Revised Rationale:

1. **Historical Context**: The 2016 election remains an outlier in terms of search interest due to its unique circumstances. However, the dynamics of the upcoming 2024 election, while different, will still play a significant role in influencing search interest. The 2020 election and subsequent events provide additional data points, supporting an expectation of high but not unprecedented search volumes.

2. **Current Context (October 2023)**: Trump’s influence in the media and political landscape is significant. Recent legal controversies and consistent media coverage ensure ongoing interest. The anticipation around his potential candidacy also maintains public and media focus.

3. **Event Timing**: As the election approaches, search interest will naturally rise. Events occurring in July 2024, such as debates, campaign rallies, and political conventions, will further escalate attention on Trump. Historical trends indicate significant spikes in search interest during such periods.

4. **Search Interest Trends**: Peaks in search interest have historically corresponded with major political events and controversies. This trend is likely to continue, with increased media focus during high-stakes political moments. Since July 2024 is close to the election, sustained interest is plausible.

5. **Other Influences**: High-profile events, such as legal proceedings or major political announcements, can dramatically increase search interest. The potential for extraordinary events adds uncertainty but increases the probability of higher search interest.

### Key Considerations:
- **Ongoing Legal Issues**: These could significantly spike search interest, especially if developments align with campaign activities.
- **Announcement of Candidacy**: If Trump officially announces his candidacy, it would spike interest.
- **Political Developments**: Engagement in key political events (debates, rallies, etc.) will maintain high levels of search interest.
- **Media Coverage**: Continuous media focus, particularly during controversies, will amplify search interest.

### Adjusted Forecast:
Considering the continued relevance of Donald Trump, the proximity to a highly anticipated election, and the potential for significant events leading to elevated search interest:

Probability: 60%

This projection acknowledges Trump's substantial ongoing impact on the political landscape, the heightened attention leading up to the 2024 election, and the potential for events that could drive substantial increases in search interest, while recognizing the uniqueness of the 2016 peak as a limiting factor.

## Critic

I have carefully reviewed the revised rationale and the sources provided. Here are my thoughts on potential flaws and areas for improvement:

1. **Historical Context**: The comparison to the 2016 Presidential Election is valid, but it might not be directly applicable to the current situation. The 2016 election was a unique event, and Trump's search interest might not follow the same pattern this time around.

2. **Current Context (October 2023)**: The rationale correctly notes that Trump remains a polarizing figure with a dedicated base and constant media presence. However, it does not fully consider the impact of his ongoing political activities and controversies on search interest.

3. **Event Timing**: The forecast correctly anticipates increased interest in Trump due to the upcoming election. However, it does not fully account for the potential impact of other events, such as new indictments or significant legal proceedings, which could significantly boost search interest.

4. **Search Interest Trends**: The rationale accurately describes the fluctuating nature of search interest in Trump. However, it does not fully explore the potential for sustained interest in Trump beyond the election period.

5. **Other Influences**: The forecast acknowledges the potential impact of extraordinary events but does not provide a detailed analysis of how these events might affect search interest.

### Key Considerations:
- **Ongoing Legal Issues**: High-profile trials or indictments could significantly boost search volumes.
- **Announcement of Candidacy**: If not yet declared, an official candidacy announcement would likely spur a surge in interest.
- **Political Developments**: Engagement in pivotal events, debates, or scandals could contribute to elevated search interest.
- **Media Coverage**: Increased media focus, especially during controversies, can greatly affect search interest.

### Adjusted Forecast Critique:
The probability of 60% seems relatively high, considering the ongoing political relevance of Donald Trump and the upcoming election. The forecast could be improved by incorporating more detailed analysis of the current political landscape and the potential impact of various events on search interest.

### Adjusted Forecast:
Probability: 65%

This revised forecast takes into account the ongoing political relevance of Donald Trump, the upcoming election, and the potential for significant events to boost search interest.


## Superforecaster

### Final Revised Rationale:

1. **Historical Context**: The 2016 Presidential Election indeed represents a dramatic peak in search interest that might not be replicable. While the comparison provides useful context, it merits noting that each election cycle has distinct dynamics. However, lessons from the 2020 election and significant events since then (like impeachment trials) provide a mixed historical basis.

2. **Current Context (October 2023)**: Trump's active role in politics and media remains a critical factor. His continuous engagement in political activities and controversies maintains a steady level of public interest. These activities can lead to periodic spikes in search interest, particularly when tied to prominent news cycles.

3. **Event Timing**: The proximity of July 2024 to the November 2024 election is a crucial factor. Historically, search interest increases as elections approach, and major political events in the months leading up can significantly drive this interest.

4. **Search Interest Trends**: Fluctuations in search interest often coincide with key events, debates, or controversies. Trends exhibit temporary peaks followed by declines, suggesting that while interest in Trump may not be sustained at 2016 levels, significant events can create comparable short-term spikes.

5. **Other Influences**: Potential extraordinary events, such as legal indictments or major political endorsements, could boost search volumes drastically. Detailed consideration of how these events overlap with election cycles enhances predictability.

### Key Considerations:
- **Ongoing Legal Issues**: Legal challenges and trials will attract media coverage, likely increasing search interest.
- **Announcement of Candidacy**: If Trump hasn't already declared his candidacy by then, any announcement will substantially spike search interest.
- **Political Developments**: Active participation in campaign rallies, primary debates, and other election-related activities will keep search interest high.
- **Media Coverage**: Extensive coverage, especially around controversies and legal issues, will fuel periodic increases.

### Adjusted Forecast:
Considering detailed insights into remaining politically active, potential for legal and political events to create spikes, and the general trend towards increased search activity in election cycles:

Probability: 65%

This refined forecast balances Trump's ongoing political relevance and the amplified interest from the election cycle against the uniquely high bar set by 2016. It incorporates the dynamic factors of legal and political developments, recognizing their significant potential to boost search interest.

## Critic

I concur.


## Daily forecast

### Get IFP ids

In [14]:
ifps = list_questions()['results']
today_ids = list(sorted([x['id'] for x in ifps]))
# today_ids = [25876, 25877, 25875, 25873, 25871, 25878, 25874, 25872] # 08JUL24
# today_ids = [26006, 25936, 25935, 25934, 25933, 26004, 26005] # 09JUL24
# today_ids = [25955, 25956, 25957, 25960, 25959, 25954, 25953, 25952, 25958] # 10JUL24
# today_ids = [26019, 26018, 26017, 26020, 26022, 26021, 26023, 26024] # 11JUL24
# today_ids = [26095, 26096, 26097, 26098, 26099, 26100, 26101, 26102] # 12JUL24
# today_ids = [26133, 26134, 26138, 26139, 26140, 26157, 26158, 26159] # 15JUL24
# today_ids = [26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196] # 16JUL24
# today_ids = [26210, 26211, 26212, 26213, 26214, 26215, 26216] # 17JUL24
# today_ids = [26232, 26233, 26234, 26235, 26236] # 18JUL24
# today_ids = [26302, 26303, 26304, 26305, 26306, 26307] # 19JUL24
# today_ids = [26387, 26388, 26389, 26390, 26391, 26392] # 22JUL24
# today_ids = [26404, 26405, 26406, 26407, 26408]

In [15]:
today_ids

[26404, 26405, 26406, 26407, 26408]

## Forecast

In [16]:
predictions = {}
for question_id in tqdm(today_ids):
    try:
        predictions[question_id]
    except:
        forecaster = Forecasting(question_id, Perplexity, ChatGPT, Perplexity)
        forecaster.forecast()
        predictions[question_id] = forecaster

  0%|                                                                       | 0/5 [00:00<?, ?it/s]

round 1


 20%|████████████▌                                                  | 1/5 [00:13<00:55, 13.80s/it]

round 1
round 2


 40%|█████████████████████████▏                                     | 2/5 [00:40<01:04, 21.41s/it]

...

 60%|█████████████████████████████████████▊                         | 3/5 [01:49<01:26, 43.15s/it]

round 1
round 2
round 3


 80%|██████████████████████████████████████████████████▍            | 4/5 [02:31<00:42, 42.59s/it]

round 1


100%|███████████████████████████████████████████████████████████████| 5/5 [02:43<00:00, 32.64s/it]


## Report

In [17]:
rpt = ""
for p in predictions.values():
    rpt += f"""
{p.report()}
===========================================================================================================
"""

display(Markdown(rpt))



# 26404 Will cannabis be removed from Schedule I of the Controlled Substance Act before September 30, 2024?



## ASSIGNMENT: 


Please assign a probability and supply a rationale for the event described by this question: [ 
The future event is described by this question: [ Will cannabis be removed from Schedule I of the Controlled Substance Act before September 30, 2024? ]
The resolution criteria are: [ This question will resolve Yes if, before September 30, 2024, marijuana is officially classified as something other than a schedule I drug in the US. If it is later reclassified as a schedule I drug, the question still resolves Yes. ]
The background is: [ According to Victoria Litman, M.Div., J.D., LL.M. writing at Harvard Law School's [Bill of Health blog](https://blog.petrieflom.law.harvard.edu/2024/05/28/what-you-need-to-know-about-marijuana-rescheduling/): 

>On May 21, 2024, the Drug Enforcement Administration (DEA) published a Notice of Proposed Rulemaking (NPRM) signed by Attorney General Merrick Garland in the Federal Register. This publication kicks off a 62-day comment period on a rule that would move marijuana to Schedule 3 of the Controlled Substances Act (CSA), classifying it as a substance with “a moderate to low potential for physical and psychological dependence.” The process of rescheduling may be long and is unlikely to create a pathway to federal compliance for state-legal marijuana businesses without further federal legislation. Ultimately, Congress likely will need to clarify the division of federal and state regulatory powers over cannabis.
 ] 

Here is some current research on this topic: [ Based on the available information, it appears that the U.S. Drug Enforcement Administration (DEA) is moving forward with a proposal to reclassify marijuana from Schedule I to Schedule III of the Controlled Substances Act (CSA). This change would categorize marijuana as a substance with a moderate to low potential for physical and psychological dependence.

The DEA published a Notice of Proposed Rulemaking (NPRM) on May 21, 2024, which initiated a 62-day public comment period. The comment period is set to end on July 22, 2024. Interested parties can submit comments electronically via the Federal eRulemaking Portal using the reference 'Docket No. DEA-1362'.

The proposed reclassification is based on a recommendation from the Department of Health and Human Services (HHS), which concluded that marijuana has a currently accepted medical use and a low potential for abuse. The DEA has not yet decided on the final scheduling of marijuana, but the agency must accord significant deference to HHS's scientific and medical determination throughout the administrative process.

While the reclassification process may take time and may not immediately create a pathway to federal compliance for state-legal marijuana businesses without further federal legislation, it marks a significant shift in federal marijuana policy. The DEA's final rule will not go into effect for 30 days after publication, during which time aggrieved parties who submitted comments can challenge the rule in court.

Given the ongoing process and the lack of a specific timeline for the final decision, it is uncertain whether cannabis will be removed from Schedule I of the Controlled Substance Act before September 30, 2024. However, the DEA's proposal and the ongoing public comment period suggest that a reclassification is likely, but the exact timing remains unclear. ]



## Superforecaster

Based on the information provided, the probability that cannabis will be removed from Schedule I of the Controlled Substances Act before September 30, 2024, involves several factors:

1. **DEA's Notice of Proposed Rulemaking (NPRM):** The DEA has already initiated the reclassification process as of May 21, 2024, and the public comment period will end on July 22, 2024. This suggests that administrative action is underway.

2. **HHS Recommendation:** The Department of Health and Human Services has recommended the reclassification, which is a significant factor since the DEA is required to give considerable weight to HHS’s determination.

3. **Timeline and Bureaucracy:** Although the process has started, the bureaucratic steps involved, including the possibility of legal challenges, could delay the final decision.

4. **Political and Social Climate:** There is growing public and political support for reclassifying marijuana due to its medicinal properties being recognized more widely.

Given these points, it seems more likely than not that cannabis will be reclassified, but the exact timing is uncertain. With the rulemaking process actively moving and significant support from HHS, but also recognizing potential delays due to further legislation and legal challenges, I will assign a probability reflective of these competing factors.

**Probability: 60%**

What is your forecast?

## Critic

I concur.


===========================================================================================================


# 26405 Will at least 5 major hurricanes occur in the 2024 Atlantic hurricane season before Sept 30, 2024?



## ASSIGNMENT: 


Please assign a probability and supply a rationale for the event described by this question: [ 
The future event is described by this question: [ Will at least 5 major hurricanes occur in the 2024 Atlantic hurricane season before Sept 30, 2024? ]
The resolution criteria are: [ This question will resolve as **Yes** if the total number of major hurricanes in the 2024 Atlantic hurricane season stands at greater than or equal to five before September 30, 2024, according to the National Hurricane Center (NHC) [website](https://www.nhc.noaa.gov/?atlc). If the number is less than five when the NHC website is accessed by Metaculus on September 30, 2024, this question resolves as **No**.

The National Weather Service [considers](https://www.weather.gov/mfl/saffirsimpson) a major hurricane to be anything Category Three or above, defined as having a 1-minute average of wind speeds greater than 110 mph (>95 knots or >177 km/hr).
 ]
The background is: [ Hurricane Beryl, a Category 5 hurricane, was the first major hurricane of the 2024 season. On July 2, 2024, it became the earliest Category 5 hurricane on record to form. Colorado State university [is forecasting](https://tropical.colostate.edu/forecasting.html) 6 major storms for 2024 as of July 19, 2024.  ]
The fine print is: [ If the official NHC page is inaccessible or there is reason to believe its data is incorrect, another resolution source may be used. ] 

Here is some current research on this topic: [ Based on the available information, the 2024 Atlantic hurricane season has already seen significant activity, with Hurricane Beryl becoming the earliest Category 5 hurricane on record in July. NOAA predicts an above-normal season with a high likelihood (85% chance) of 17-25 named storms, 8-13 hurricanes, and 4-7 major hurricanes. 

As of July 22, the season's ACE index stands at approximately 36.1 units, indicating a high level of activity. The National Weather Service forecasts 17-25 named storms, with 8-13 of those becoming hurricanes, and 4-7 major hurricanes. 

Colorado State University is also forecasting 6 major storms for 2024 as of July 19. While these predictions do not guarantee that at least 5 major hurricanes will occur before September 30, they do suggest a high probability of an active season. 

Given the current trends and predictions, it is likely that at least 5 major hurricanes will occur in the 2024 Atlantic hurricane season before September 30, 2024. ]



## Superforecaster

Given the background information, the likelihood of having at least 5 major hurricanes in the 2024 Atlantic hurricane season before September 30, 2024, seems quite high. Several factors point to this:

1. **Current Activity**: The already significant activity, with Hurricane Beryl becoming the earliest Category 5 hurricane on record, is a strong indicator of an active season.
2. **Forecasts**: Both NOAA and Colorado State University project an above-normal season with high numbers of major hurricanes. NOAA forecasts 4-7 major hurricanes with an above-normal season likelihood of 85%, and Colorado State's specific prediction is 6 major storms.
3. **Historical Data**: Based on past active seasons and current trends (ACE index), it seems plausible that the activity will sustain or even increase in the coming months.

Putting together these factors suggests a high probability of meeting the threshold of 5 major hurricanes before September 30, 2024.

### Probability: 75%

What do you think?

## Critic

While the factors presented do suggest an active season, I have some reservations about the probability of having at least 5 major hurricanes before September 30, 2024. Here are my concerns:

1. **Current Activity**: While Hurricane Beryl's early formation and rapid intensification are significant, it is essential to consider that individual storms do not necessarily dictate the overall season. Other factors, such as the transition from El Niño to La Niña conditions, are more critical in shaping the broader hurricane activity.

2. **Forecasts**: Although both NOAA and Colorado State University predict an above-normal season, their specific predictions are not identical. NOAA forecasts 4-7 major hurricanes, while Colorado State predicts 6. This disparity highlights the uncertainty involved in these forecasts. Additionally, the confidence levels of these predictions are not explicitly stated, which could impact the reliability of the forecasts.

3. **Historical Data**: While historical trends and the ACE index do suggest an active season, it is crucial to recognize that each season is unique and influenced by various factors. The current season's activity, though significant, might not necessarily translate to sustained or increased activity in the coming months.

Considering these points, I would revise the probability to 60%.


## Superforecaster

You raise valid concerns which merit a more nuanced evaluation. Your reservations about the role of individual storms, the non-identical forecasts, and the uncertainty inherent in weather predictions are important to consider.

1. **Current Activity**: Indeed, while Hurricane Beryl's early formation is notable, individual events don't guarantee the overall trend. The influence of broader climatic conditions, such as the potential transition from El Niño to La Niña, can significantly impact storm formation rates and intensities during the season, introducing variability.

2. **Forecasts**: The forecasts from NOAA and Colorado State University, while indicative of an active season, differ slightly, emphasizing forecasting uncertainty. While NOAA's prediction range (4-7 major hurricanes) overlaps with Colorado State University's specific figure (6 major hurricanes), this still introduces some variance.

3. **Historical Data**: The ACE index and historical trends indeed suggest a high level of activity, but unique seasonal factors and unpredictable events also play significant roles. The high variability year-on-year in hurricane seasons makes projection harder.

Taking your points into account, it would make sense to slightly lower the probability while keeping it in the range that reflects a still likely but not certain outcome.

### Revised Probability: 65%

Does this seem more balanced?

## Critic

I concur.


===========================================================================================================


# 26406 Will the maximum weekly rate of COVID-19 hospitalizations occurring in the United States between June 1 and August 31, 2024 exceed 3.5 per 100,000 people?



## ASSIGNMENT: 


Please assign a probability and supply a rationale for the event described by this question: [ 
The future event is described by this question: [ Will the maximum weekly rate of COVID-19 hospitalizations occurring in the United States between June 1 and August 31, 2024 exceed 3.5 per 100,000 people? ]
The resolution criteria are: [ The question will resolve as **Yes** if the maximum weekly rate of hospital admissions per 100,000 people for COVID-19 between the weeks ending June 1, 2024, and August 31, 2024, (inclusive) exceeds 3.5 according to the CDC's [RESP-NET](https://www.cdc.gov/surveillance/resp-net/dashboard.html). The CDC uses "epidemiological weeks" which begin on Sunday and end on Saturday. The rate used will be that for all ages, races, and sexes. See the fine print for the correct filters.

To allow for data revisions and reporting delays, the question will resolve according to data shown after the reporting delay period, when the figure for August 31, 2024, is not shown as a dashed line, typically 4 weeks after the week end date. ]
The background is: [ A [new COVID variant is on the rise](https://www.cbsnews.com/news/flirt-covid-variants-us-cases/) in the United States, variant KP.2, estimated at 28.2% of sequenced variants for the two-week period ending May 11, 2024, according to the CDC's [variant proportions Nowcast](https://covid.cdc.gov/covid-data-tracker/#variant-proportions). The KP.2 variant is one of a set of new variants [nicknamed](https://twitter.com/tryangregory/status/1767884323938869586?t=NM6QgCzjQYBje9-ZtyxSvg) as the FLiRT variants after the F456L + R346T spike protein where this variant shows changes. The new variants have raised concerns of a potential [summer COVID surge](https://www.webmd.com/covid/news/20240430/new-variant-sparks-summer-covid-surge-warning).

The US Centers for Disease Control and Prevention (CDC) closely monitors respiratory illnesses, particularly when these illnesses increase in prevalence during the winter. One monitoring and data sharing tool the CDC uses is [RESP-NET](https://www.cdc.gov/surveillance/resp-net/dashboard.html), which it describes as follows:

>The Respiratory Virus Hospitalization Surveillance Network (RESP-NET) comprises three networks that conduct population-based surveillance for laboratory-confirmed hospitalizations associated with COVID-19, respiratory syncytial virus (RSV), and influenza among children and adults. While RESP-NET does not collect data on all hospitalizations caused by respiratory illnesses, it can describe hospitalizations caused by three viruses that account for a large proportion of these hospitalizations. Surveillance is conducted through a network of acute care hospitals in select counties or county equivalents in 12 states for RSV surveillance, 13 states for COVID-19 surveillance, and 14 states for influenza surveillance. The surveillance platforms for COVID-19, RSV, and influenza (known as [COVID-NET](https://www.cdc.gov/coronavirus/2019-ncov/covid-data/covid-net/purpose-methods.html), [RSV-NET](https://www.cdc.gov/rsv/research/rsv-net/overview-methods.html), and [FluSurv-NET](https://www.cdc.gov/flu/weekly/influenza-hospitalization-surveillance.htm), respectively) cover more than 30 million people and include an estimated 8-10% of the U.S. population.

The underlying source data for the RESP-NET dashboard can be found at [this link](https://data.cdc.gov/Public-Health-Surveillance/Rates-of-Laboratory-Confirmed-RSV-COVID-19-and-Flu/kvib-3txy). ]
The fine print is: [ * The following filters will be selected:

    * **Chart Selection:** "Season"
    * **View:** "Weekly Rates"
    * **Season:** "2023-24" (or whichever selections will encompass the dates in question)
    * **Pathogen:** "All" (Note that you can also filter to only show the pathogen of interest. Regardless, only the value corresponding to the listed pathogen will be used.)
* This question is asking about the global maximum weekly rate, the highest value reached during the specified period.
* In the event the data reporting is missing for more than two of the weeks included in the period when the question is expected to resolve, the question will be **annulled**. ] 

Here is some current research on this topic: [ The future event in question is whether the maximum weekly rate of COVID-19 hospitalizations in the United States between June 1 and August 31, 2024, will exceed 3.5 per 100,000 people. The Centers for Disease Control and Prevention (CDC) closely monitors respiratory illnesses, including COVID-19, through the Respiratory Virus Hospitalization Surveillance Network (RESP-NET).

### Key Points:
1. **Current Trends:** As of May 10, 2024, the CDC reported the lowest number of new weekly coronavirus hospitalizations since the pandemic began, with approximately 5,600 new hospitalizations during the week ending April 20, 2024.
2. **New Variant:** A new COVID-19 variant, KP.2, is on the rise in the United States, accounting for 28.2% of sequenced variants for the two-week period ending May 11, 2024.
3. **Monitoring Tools:** The CDC uses RESP-NET to track laboratory-confirmed hospitalizations associated with COVID-19, RSV, and influenza among children and adults. This network covers more than 30 million people and includes an estimated 8-10% of the U.S. population.
4. **Data Reporting:** The CDC's RESP-NET dashboard provides weekly rates of hospitalizations, which can be filtered by pathogen, age group, race and ethnicity, sex, season, and state.
5. **Resolution Criteria:** The question will resolve as **Yes** if the maximum weekly rate of hospital admissions per 100,000 people for COVID-19 between the weeks ending June 1, 2024, and August 31, 2024, exceeds 3.5 according to the CDC's RESP-NET dashboard.

### Summary:
Given the current downward trend in hospitalizations and the emergence of the new KP.2 variant, the likelihood of the maximum weekly rate of COVID-19 hospitalizations exceeding 3.5 per 100,000 people between June 1 and August 31, 2024, is uncertain. The CDC's RESP-NET dashboard will be used to track and determine the resolution of this question. ]



## Superforecaster

### Rationale:

1. **Current Trends:** The overall trend in COVID-19 hospitalizations has been decreasing. As of May 2024, the CDC reported the lowest weekly hospitalizations since the pandemic began.
2. **New Variant:** The new KP.2 variant is on the rise, but its actual impact on hospitalizations remains to be seen. While it accounted for 28.2% of sequenced variants, there is no clear evidence yet suggesting it significantly increases hospitalization rates compared to previous variants.
3. **Past Data:** Historical data may provide some context. For instance, during certain spikes, weekly hospitalization rates did exceed 3.5 per 100,000, but these were often associated with highly virulent strains or periods of low vaccine effectiveness.
4. **Vaccination and Immunity:** With a significant portion of the population either vaccinated or previously infected, it remains to be seen whether this immunity can curtail the rise in hospitalizations.
5. **CDC Monitoring and Reporting:** The robust RESP-NET system ensures accurate and timely data, reducing the uncertainty in data collection and reporting.

Given these factors, I find a moderate probability for the event considering the new variant but also considering the overall trend and existing immunity.

### Forecast:
Probability: 45%



## Critic

I have reviewed the provided sources and the rationale for the forecast. Here are my thoughts on the potential flaws in the rationale:

1. **Current Trends:** While it is true that hospitalizations have been decreasing, the article from U.S. News highlights that this trend might be disrupted by new variants. The current trend might not be a reliable indicator for future hospitalizations.

2. **New Variant:** The KP.2 variant is gaining prevalence, but its impact on hospitalizations is still unclear. The lack of conclusive evidence on its severity might be a significant factor in the forecast.

3. **Past Data:** Historical data can be informative, but it might not be directly applicable to the current situation. The effectiveness of vaccines and immunity levels have changed over time, which could affect the impact of new variants.

4. **Vaccination and Immunity:** The high level of immunity in the population is a positive factor, but it is unclear how well this immunity will hold against new variants. The CDC emphasizes the importance of staying up to date with vaccinations, which suggests that existing immunity might not be sufficient.

5. **CDC Monitoring and Reporting:** While the RESP-NET system is robust, the shift in hospitalization reporting requirements might lead to gaps in data collection, which could impact the accuracy of forecasts.

Considering these points, I believe the forecast probability of 45% might be overly optimistic. The uncertainty surrounding the new variant and the potential impact of changing immunity levels could lead to a higher probability of increased hospitalizations.


## Superforecaster

### Revised Rationale:

1. **Current Trends:** While the past trend shows a decrease in hospitalizations, the emergence of new variants, such as KP.2, has historically disrupted such trends. The potential for the KP.2 variant to change the current trend must be taken seriously.

2. **New Variant:** The lack of conclusive evidence on the KP.2 variant’s severity introduces significant uncertainty. If this variant has higher transmissibility or partial vaccine escape, it could impact hospitalization rates more severely than anticipated. Given that the variant has already reached 28.2% of sequenced cases, its impact is worth considering more heavily.

3. **Past Data:** The historical peaks in hospitalizations during periods of variant surges provide a precedent, even if exact conditions differ. Understanding that these spikes are plausible adds weight to the probability of exceeding the 3.5 per 100,000 threshold.

4. **Vaccination and Immunity:** While high levels of immunity due to vaccination and past infections are beneficial, the waning of immunity and potential for immune escape mutations in new variants mean that it is not a guaranteed safeguard against future spikes in hospitalizations. The CDC's reminder to stay updated with vaccinations indicates an ongoing concern about the durability of existing immunity.

5. **CDC Monitoring and Reporting:** While the RESP-NET system is robust, changes in hospitalization reporting requirements may introduce variability in data accuracy. This factor can either overestimate or underestimate actual hospitalization rates but needs to be considered as a potential complicating factor.

Given these revised considerations, the probability of the maximum weekly rate of COVID-19 hospitalizations exceeding 3.5 per 100,000 people between June 1 and August 31, 2024, should be adjusted upward to account for the increased uncertainty and potential impact of the new variant.

### Updated Forecast:
Probability: 60%

## Critic

I have reviewed the revised rationale and the sources provided. Here are my thoughts on the potential flaws in the revised rationale:

1. **Current Trends:** The revised rationale correctly acknowledges the potential disruption of the current trend by the emergence of new variants. However, it might be beneficial to consider the recent increase in hospitalizations in New York State, which could be an early indication of the impact of the KP.2 variant.

2. **New Variant:** The revised rationale correctly emphasizes the uncertainty surrounding the KP.2 variant’s severity and potential for immune escape. However, it might be helpful to consider the similarities between the KP.2 variant and other variants like JN.1, which had milder symptoms.

3. **Past Data:** The revised rationale appropriately references historical peaks in hospitalizations during variant surges. However, it is crucial to consider the differences in vaccination rates and immunity levels between past surges and the current situation.

4. **Vaccination and Immunity:** The revised rationale correctly notes the waning of immunity and the potential for immune escape mutations. However, it might be beneficial to consider the ongoing efforts to update vaccinations and the CDC’s encouragement to stay current with vaccinations, which could help mitigate the impact of new variants.

5. **CDC Monitoring and Reporting:** The revised rationale correctly acknowledges the potential impact of changes in hospitalization reporting requirements on data accuracy. However, it is essential to consider the CDC’s continued efforts to monitor and report COVID-19 data, including the use of wastewater surveillance and voluntary reporting mechanisms.

Considering these points, I believe the updated forecast probability of 60% might still be somewhat conservative. The uncertainty surrounding the KP.2 variant and the potential impact of changing immunity levels could lead to a higher probability of exceeding the 3.5 per 100,000 threshold.


## Superforecaster

### Revised Rationale:

1. **Current Trends:** Acknowledging the disruption potential of the KP.2 variant, it is also crucial to consider specific regional data indicating early impacts. The recent increase in hospitalizations in New York State, for instance, could be an early indicator suggesting more widespread effects.

2. **New Variant:** While the KP.2 variant has similarities with milder variants like JN.1, the uncertainty surrounding its potential for immune escape and higher transmissibility continues to be a significant concern. Variants spreading rapidly tend to have impactful outcomes on hospitalizations, which cannot be entirely dismissed based on historical mild variants alone.

3. **Past Data:** The differences in vaccination rates and immunity levels must be accounted for, but these differences do not negate the historical data showing hospitalization surges during new variant waves. Even with higher immunity now, the phenomenon of breakthrough infections and variant-driven hospitalizations cannot be overlooked.

4. **Vaccination and Immunity:** The ongoing efforts to update vaccinations and the CDC's encouragement are positive steps, but the impact of these efforts may take time to materialize. Meanwhile, the potential for immune escape in new variants suggests that the protection conferred may not be absolute. This nuance indicates a tangible risk that must be reflected in the probability.

5. **CDC Monitoring and Reporting:** The CDC’s adaptive strategies, such as wastewater surveillance and voluntary reporting, enhance data reliability even amidst reporting requirement changes. These adaptive measures suggest the CDC will likely capture real hospitalization trends accurately, supporting confident modeling of potential hospitalization spikes.

Given these enhanced considerations, the probability of the maximum weekly rate of COVID-19 hospitalizations exceeding 3.5 per 100,000 people between June 1 and August 31, 2024, should be set higher to reflect the complex interplay of emerging variants, current immunity, and real-time data reporting.

### Updated Forecast:
Probability: 70%

## Critic

I have reviewed the revised rationale and the sources provided. Here are my thoughts on the potential flaws in the revised rationale:

1. **Current Trends:** The revised rationale correctly acknowledges the potential disruption of the current trend by the emergence of new variants. The recent increase in hospitalizations in New York State is a valid indicator of the potential impact of the KP.2 variant. However, it might be beneficial to consider the broader national trend, which shows a steady rise in hospitalizations since April, as reported by the CDC.

2. **New Variant:** The revised rationale correctly emphasizes the uncertainty surrounding the KP.2 variant’s potential for immune escape and higher transmissibility. While it is true that the KP.2 variant shares similarities with milder variants like JN.1, the lack of conclusive evidence on its severity and potential impact on hospitalizations remains a significant concern.

3. **Past Data:** The revised rationale appropriately references historical peaks in hospitalizations during variant surges. However, it is crucial to consider the differences in vaccination rates and immunity levels between past surges and the current situation. The ongoing efforts to update vaccinations and the CDC’s encouragement to stay current with vaccinations might help mitigate the impact of new variants, but the potential for immune escape and breakthrough infections cannot be entirely dismissed.

4. **Vaccination and Immunity:** The revised rationale correctly notes the ongoing efforts to update vaccinations and the CDC’s encouragement. However, it might be beneficial to consider the current vaccination rates, which remain low for both children and adults, as reported by the CDC. This could impact the effectiveness of vaccination efforts in preventing hospitalizations.

5. **CDC Monitoring and Reporting:** The revised rationale correctly acknowledges the CDC’s adaptive strategies, such as wastewater surveillance and voluntary reporting, which enhance data reliability. However, it is essential to consider the potential impact of changes in hospitalization reporting requirements on data accuracy, as noted by the CDC.

Considering these points, I believe the updated forecast probability of 70% might still be somewhat conservative. The uncertainty surrounding the KP.2 variant, the recent increase in hospitalizations in New York State, and the ongoing efforts to update vaccinations all contribute to a higher probability of exceeding the 3.5 per 100,000 threshold.


## Superforecaster

### Final Rationale:

1. **Current Trends:** Beyond the initial increase in New York State, the broader national trend shows a steady rise in hospitalizations since April, as reported by the CDC. This ongoing national trend suggests a generalized upward trajectory that could be exacerbated by the growing prevalence of the KP.2 variant.

2. **New Variant:** While the KP.2 variant displays similarities to milder variants like JN.1, the uncertainty about its severity, immune escape potential, and higher transmissibility continues to weigh heavily. The absence of conclusive data on its impact means we must prepare for scenarios where it could significantly boost hospitalizations.

3. **Past Data:** Historical peaks in hospitalizations during new variant surges do provide a strong precedent. The differences in current vaccination rates and immunity levels must be acknowledged, but so must the potential for immune escape and the realities of breakthrough infections. These factors are still critical in forecasting potential hospitalization surges.

4. **Vaccination and Immunity:** The ongoing efforts to update vaccinations and maintain immunity are beneficial, but current vaccination rates, particularly among children and adults, remain low according to the CDC. This low uptake could diminish the overall community protection and increase vulnerability to the KP.2 variant's impact.

5. **CDC Monitoring and Reporting:** The CDC’s adaptive strategies, including wastewater surveillance and voluntary reporting, bolster data reliability despite potential changes in hospitalization reporting requirements. These measures mean that real-time trends can still be accurately captured and used for predictive modeling.

Considering the broader rise in nationwide hospitalizations, the growing prevalence of the KP.2 variant, uncertain immunity dynamics, and current vaccination rates, a higher forecast probability is warranted.

### Final Forecast:
Probability: 80%

## Critic

I have reviewed the final rationale and the sources provided. Here are my thoughts on the potential flaws in the final rationale:

1. **Current Trends:** The final rationale correctly acknowledges the broader national trend of a steady rise in hospitalizations since April, as reported by the CDC. This ongoing trend supports the forecast of increased hospitalizations.

2. **New Variant:** The final rationale correctly emphasizes the uncertainty surrounding the KP.2 variant’s severity, immune escape potential, and higher transmissibility. The lack of conclusive data on its impact means that it is prudent to prepare for scenarios where it could significantly boost hospitalizations.

3. **Past Data:** The final rationale appropriately references historical peaks in hospitalizations during new variant surges. However, it is crucial to consider the differences in current vaccination rates and immunity levels compared to past surges. The ongoing efforts to update vaccinations and maintain immunity are beneficial, but the potential for immune escape and breakthrough infections must be acknowledged.

4. **Vaccination and Immunity:** The final rationale correctly notes the low vaccination rates, particularly among children and adults, as reported by the CDC. This low uptake could diminish the overall community protection and increase vulnerability to the KP.2 variant’s impact.

5. **CDC Monitoring and Reporting:** The final rationale correctly acknowledges the CDC’s adaptive strategies, including wastewater surveillance and voluntary reporting, which bolster data reliability despite potential changes in hospitalization reporting requirements. These measures mean that real-time trends can still be accurately captured and used for predictive modeling.

Considering the broader rise in nationwide hospitalizations, the growing prevalence of the KP.2 variant, uncertain immunity dynamics, and current vaccination rates, I concur with the final forecast probability of 80%.


===========================================================================================================


# 26407 Will Russia start a second mobilization wave before September 30, 2024, according to the Institute for the Study of War?



## ASSIGNMENT: 


Please assign a probability and supply a rationale for the event described by this question: [ 
The future event is described by this question: [ Will Russia start a second mobilization wave before September 30, 2024, according to the Institute for the Study of War? ]
The resolution criteria are: [ This question will be resolved as **Yes** if, before September 30, 2024, the [Institute for the Study of War](https://www.understandingwar.org/backgrounder/ukraine-conflict-updates) (ISW) reports that a second wave of mobilization is underway in Russia. If this event does not happen, this question will resolve as **No**. ]
The background is: [ According to the ISW: 

> ISW continues to assess that Putin is likely aware that a second mobilization wave would be widely unpopular and is concerned that such a measure would generate widespread discontent

[RUSSIAN OFFENSIVE CAMPAIGN ASSESSMENT](https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-25-2024), February 25, 2024.

[According to Wikipedia:](https://en.wikipedia.org/wiki/2022_Russian_mobilization)

> On 21 September 2022, seven months into the Russian invasion of Ukraine, Russia declared a partial mobilization of military reservists. The decision was made a day after the announcement of the Russian annexation of the DPR, LPR, Kherson and Zaporizhzhia oblasts.
> The announcement of mobilization was seen as a significant escalation of Russia's military efforts in the war with Ukraine. Defense Minister Sergei Shoigu announced that Russia had a "huge mobilization reserve" and planned to mobilize 300,000 recruits. The precise details of the mobilization plans are currently unclear, however, as the exact number of people to be mobilized is classified.
> On 28 October, Shoigu told Russian president Vladimir Putin that mobilization had been completed, which was followed an announcement by Putin of its completion.

As of this question's writing, there is a lot of uncertainty about second wave of mobilization. On one hand, [Russia suffered staggering losses](https://www.metaculus.com/questions/11243/russian-military-deaths-by-2024/), and Ukraine plans to [mobilize 500'000 more troops](https://www.reuters.com/world/europe/ukraines-military-asks-additional-450000-500000-people-be-mobilised-zelenskiy-2023-12-19/) this year. On the other hand, [Russia probably recruited 400'000 troops last year](https://t.me/medvedev_telegram/402) and plans [to do the same this year](https://www.themoscowtimes.com/2023/12/22/how-russian-officials-plan-to-recruit-400k-new-contract-soldiers-in-2024-a83509).


Related Question on Metaculus: [Will Russia start another wave of mobilization before the following dates?](https://www.metaculus.com/questions/14006/russia-2nd-wave-of-mobilization) ]
The fine print is: [ - For this resolution, "mobilization" is defined as the formal conscription or summoning of reservists by the Russian government, as explicitly recognized and reported by ISW. This includes actions officially declared by the government such as a "partial mobilization." However, this does not include unofficial recruitments often referred to as "[crypto-mobilization](https://politics.stackexchange.com/questions/75547/what-is-crypto-mobilization)" or "self-mobilization," or any form of volunteer enlistment. The resolution will rely solely on ISW’s characterization of the mobilization as formally acknowledged by the Russian government.

- If the Institute for the Study of War writes that the next wave of mobilization has not only begun, but has already ended, then this also resolves as Yes.

- ISW will be the sole source used for resolution of this question. If ISW ceases to operate or ceases to publish the relevant updates following the launch of this question, it may resolve as Annulled. Metaculus will determine appropriate resolution based on its assessment of ISW's publications. However, unless ISW’s coverage on the topic is discontinued in this manner, if no official mobilization is reported by ISW before September 30, 202, then this question will resolve as No.
 ] 

Here is some current research on this topic: [ The question asks whether Russia will start a second mobilization wave before September 30, 2024, according to the Institute for the Study of War (ISW). The resolution criteria are based on ISW's reports, and the question will be resolved as "Yes" if ISW reports a second wave of mobilization before September 30, 2024, and "No" if it does not.

As of July 24, 2024, there is no definitive information on a second mobilization wave. However, several sources suggest that Russia is preparing for further mobilization efforts:

1. **Russian Mobilization and Force Generation Efforts**: A Russian milblogger reported that wounded soldiers are being sent back to the front without proper medical clearance, indicating that Russia is trying to expand its combat power without conducting a general mobilization.
2. **Russian State Duma**: Russian State Duma Deputy Yaroslav Nilov stated that he would request the Russian Ministry of Defense to resolve the issue of wounded soldiers being sent back to the front.
3. **Kremlin Meeting**: *Meduza*'s sources suggested that Kremlin officials do not fully understand the scale of the situation and are discussing potential mobilization measures.

While these reports indicate that Russia is taking steps to increase its military presence, there is no concrete evidence of a second mobilization wave. The ISW continues to assess that a second mobilization wave would be widely unpopular and might generate widespread discontent. Therefore, as of July 24, 2024, the answer to this question remains uncertain and will depend on future ISW reports. ]



## Superforecaster

Given the available information, we can analyze several factors that could influence the likelihood of Russia initiating a second mobilization wave:

1. **Historical Context**: The first mobilization wave in September 2022 aimed to recruit 300,000 reservists. This move was seen as a significant escalation and faced domestic backlash.

2. **Current Military Needs**: Russia has suffered considerable losses in the ongoing conflict in Ukraine, and its combat readiness is likely strained. If Ukraine follows through with its planned mobilization of 500,000 more troops, Russia might feel pressured to respond similarly to maintain its military presence and capabilities.

3. **Recruitment Efforts**: Reports indicate that Russia recruited around 400,000 troops last year and plans to do the same this year. If successful, the need for a second formal mobilization might be mitigated.

4. **Public Sentiment and Political Calculations**: According to ISW, Russian leadership, including Putin, is acutely aware that another wave of mobilization could be highly unpopular and risk internal instability. The government’s approach might prefer covert or less formal methods of troop reinforcement (like crypto-mobilization) to avoid public outrage.

5. **ISW Assessment**: The resolution depends on whether ISW explicitly reports a second wave of mobilization. As of now, there is no concrete information suggesting that such an event is imminent.

Considering these factors, my initial forecast is predicated on the balancing act Russia must maintain between its military needs and domestic stability:

**Probability: 30%**

This reflects a moderately low likelihood that Russia will resort to a formal second wave of mobilization due to potential domestic backlash, although the escalating military demands could push the leadership to take such a step.


## Critic

I disagree with the forecast. Here are my concerns and alternative perspectives:

1. **Historical Context**: While the first mobilization wave faced domestic backlash, it is crucial to consider the evolving context. Russia has since adapted to the ongoing conflict and might have learned from its previous experiences. This could lead to more effective and less controversial mobilization strategies.

2. **Current Military Needs**: The significant losses Russia has incurred in Ukraine underscore the need for replenishing its military strength. Given the ongoing conflict and the potential for further escalation, Russia might feel compelled to initiate a second mobilization wave to maintain its military capabilities.

3. **Recruitment Efforts**: Although Russia has made efforts to recruit troops, the scale of the ongoing conflict and the losses incurred suggest that these efforts might not be sufficient to meet the military's needs. A second formal mobilization wave could be necessary to ensure Russia's military readiness.

4. **Public Sentiment and Political Calculations**: While public sentiment is a crucial factor, it is essential to consider the Russian government's ability to manage and manipulate public opinion. Putin's leadership has demonstrated a capacity to maintain control and suppress dissent. This could enable the government to implement a second mobilization wave without significant internal instability.

5. **ISW Assessment**: The lack of concrete information about a second wave of mobilization does not necessarily imply that it is unlikely. Russia might be planning a covert or less formal mobilization, which could be more difficult to detect.

Considering these factors, I believe the probability of Russia initiating a second mobilization wave is higher than the initial forecast:

**Probability: 50%**

This reflects a more balanced assessment, acknowledging both the potential domestic backlash and the pressing military needs that could drive Russia to take this step.


## Superforecaster

You raise valid points that highlight the dynamic and multifaceted nature of this issue. Let's reassess the probability by weighing the additional perspectives you've provided.

1. **Adaptation and Strategic Learning**: The Russian government likely has gained insights from the first wave of mobilization. It may have developed strategies to mitigate the backlash, potentially through more efficient and less controversial methods of conscription.

2. **Pressing Military Needs**: Given the ongoing conflict and the substantial losses incurred, replenishing military ranks is a pressing necessity. The sheer scale of warfare might push Russia to undertake a formal mobilization to meet tactical demands.

3. **Sufficiency of Recruitment Efforts**: The ongoing recruitment initiatives may fall short of the required numbers, driving the need for a formal second mobilization wave to bolster the military's operational capacity significantly.

4. **Control Over Public Sentiment**: The Russian government has extensive mechanisms to manage and suppress dissent, which might facilitate the implementation of another mobilization wave without causing severe internal instability.

5. **Planning and Reporting by ISW**: The possibility of covert or less formal mobilization is significant. However, our focus is on whether a second wave is formally acknowledged and reported as such by ISW. The potential for this to happen is influenced by shifting military strategies and intelligence assessments.

Given this broader set of considerations, which balances the likelihood of Russia needing additional troops with the potential capacity to organize another wave while managing public opinion:
 
**Probability: 45%**

This refined probability reflects an increased likelihood from the initial assessment, incorporating the potential for Russia to undertake a second mobilization wave, tempered by the challenges of achieving it without significant domestic repercussions.

## Critic

I disagree with the revised forecast. Here are my concerns and alternative perspectives:

1. **Adaptation and Strategic Learning**: While Russia has likely learned from the first mobilization wave, it is crucial to consider the evolving context. The ongoing conflict and the significant losses incurred might necessitate more drastic measures to replenish military strength.

2. **Pressing Military Needs**: The sheer scale of warfare and the substantial losses Russia has incurred underscore the need for replenishing military ranks. This could push Russia to undertake a formal mobilization to meet tactical demands.

3. **Sufficiency of Recruitment Efforts**: The ongoing recruitment initiatives might fall short of the required numbers, driving the need for a formal second mobilization wave to bolster the military's operational capacity significantly.

4. **Control Over Public Sentiment**: The Russian government has extensive mechanisms to manage and suppress dissent, which might facilitate the implementation of another mobilization wave without causing severe internal instability.

5. **Planning and Reporting by ISW**: The possibility of covert or less formal mobilization is significant. However, our focus is on whether a second wave is formally acknowledged and reported as such by ISW. The potential for this to happen is influenced by shifting military strategies and intelligence assessments.

Considering these factors, I believe the probability of Russia initiating a second mobilization wave is higher than the revised forecast:

**Probability: 60%**

This reflects a more balanced assessment, acknowledging both the pressing military needs and the potential capacity to organize another wave while managing public opinion.


## Superforecaster

You've presented strong arguments that highlight the urgency and potential inevitability of a second mobilization wave given the strategic context. Let's revisit the probability with this enhanced analysis:

1. **Evolving Context and Strategic Learning**: While Russia has indeed faced substantial resistance and backlash, the necessity to adapt and address current military deficiencies might outweigh the concerns learned from past experiences. This could drive the military leadership to pursue more immediate and effective solutions, potentially through formal mobilization.

2. **Intensity of Military Needs**: The ongoing conflict and extensive losses indeed require substantial troop replenishments. Given the demands of maintaining military pressure and operational capacity, a formal mobilization can become a compelling option.

3. **Insufficiency of Current Recruitment Efforts**: Even with ongoing recruitment, the scale of the conflict might necessitate a more systematic approach to troop mobilization. If current efforts fail to meet targets, a formal mobilization wave could be plausible.

4. **Government's Control Over Public Opinion**: The government's ability to manage dissent and control the narrative is a crucial factor. This control could potentially mitigate the risks of widespread discontent, thereby lowering the barriers to formal mobilization.

5. **ISW Reporting**: The focus on formal mobilization recognized by ISW requires tangible, explicit actions by the Russian government. As the strategic situation evolves, ISW's intelligence assessments and reporting will play a critical role in identifying such developments.

Given the imperative of military needs and the Russian government’s ability to manage internal resistance effectively:
 
**Probability: 55%**

This updated probability reflects a higher likelihood, acknowledging the pressing military requirements and the viable capacity of the Russian government to implement another mobilization wave while managing public opinion. This also aligns closely with your argument that the combination of military exigencies and strategic considerations make a formal mobilization wave more probable.

## Critic

I concur.


===========================================================================================================


# 26408 Will the minimum wage in Türkiye (Turkey) increase again before September 30, 2024?




## ASSIGNMENT: 


Please assign a probability and supply a rationale for the event described by this question: [ 
The future event is described by this question: [ Will the minimum wage in Türkiye (Turkey) increase again before September 30, 2024?
 ]
The resolution criteria are: [ This question will resolve as **Yes** if [credible sources](https://www.metaculus.com/help/faq/#definitions) report before September 30, 2024, that another minimum wage increase was going to take effect in Turkey in 2024. Wage increases that are approved before September 30, 2024, but taking effect in 2025 (e.g., starting January, 1, 2025) do not count towards resolution. If no such report is made before September 30, 2024, this question resolves as **No**.
 ]
The background is: [ In recent years, Turkey experienced high inflation, climbing [up to 85%](https://tradingeconomics.com/turkey/inflation-cpi) at peak, and 68% year-on-year as of April, 2024. Due to high inflation, Turkish government diverged from its policy of yearly increases in minimum wage, and increased minimum wage twice in [2022 and 2023](https://en.wikipedia.org/wiki/Minimum_wage_in_Turkey). The last minimum wage increase was [on January, 1, 2024](https://www.reuters.com/world/middle-east/turkey-raises-monthly-minimum-wage-by-49-2024-2023-12-27/), when wages rose by 50%. However, it was already [partially eaten by inflation](https://stockholmcf.org/turkeys-hunger-threshold-exceeds-minimum-wage/). The share of workers receiving minimum wage is estimated to be between [33% and 50%](https://www.intellinews.com/turkey-s-minimum-wage-becoming-average-wage-protest-unions-304930/#:~:text=Around%2050%25%20of%20workers%20in,to%20DISK%20chair%20Arzu%20Cerkezoglu.).

Recently the ruling party of Turkish president Recep Tayyip Erdoğan lost in local elections, in what has been described as a "[historic victory](https://foreignpolicy.com/2024/04/01/turkey-elections-chp-imamoglu-erdogan-akp-israel-iran-consulate-syria/)" for the opposition. There has been speculation that this might spark populist policies such as an increased minimum wage, but in April 2024 the Turkish Labor and Social Security Minister Vedat Işıkhan [said](https://www.duvarenglish.com/turkish-labor-minister-rules-out-second-minimum-wage-hike-in-2024-news-64217) about a possible second minimum wage hike, "We have no such agenda. We will continue as we decided at the beginning." ]
The fine print is: [ Increases of any size, no matter how small, count towards the resolution of this question. ] 

Here is some current research on this topic: [ Based on the available sources, there is no indication that the minimum wage in Türkiye (Turkey) will increase again before September 30, 2024. The last minimum wage increase was on January 1, 2024, with a 49% hike to TRY 20,002.50 (gross) and TRY 17,002.12 (net). The Turkish Labor and Social Security Minister Vedat Işıkhan has explicitly stated that there are no plans for a second minimum wage hike in 2024. ]



## Superforecaster

### Initial Forecast

**Rationale:**
1. **Historical Precedent**: Turkey has deviated from its annual minimum wage increase policy in recent years due to high inflation, opting for multiple increases within a year (twice in 2022 and 2023).
2. **Current Economic Context**: With ongoing high inflation (68% year-on-year as of April 2024), there is continued erosion of purchasing power, which may necessitate another wage adjustment to maintain stability and public support.
3. **Political Factors**: The ruling party recently faced a historic defeat in local elections, which could pressure them to adopt populist measures like another minimum wage hike to regain popularity ahead of future elections.
4. **Official Statements**: Despite the recent statement from the Turkish Labor and Social Security Minister Vedat Işıkhan ruling out a second increase in 2024, political and economic pressures could cause a shift in policy.
5. **Worker Demographics**: A significant portion of the workforce (33% to 50%) is dependent on minimum wage, increasing the likelihood of another wage hike to appease a large voter base.

**Probability:** 65% 

### Feedback
My friend might say: "The Turkish Labor Minister's recent statement explicitly rules out another hike this year. Doesn't this significantly decrease the likelihood?"

### Reconsidered Forecast

**Revised Rationale:**
1. **Official Denial**: The Minister's explicit statement ruling out another wage hike does lower the probability somewhat, as it represents the current official stance.
2. **Flexibility of Political Statements**: However, political statements can change quickly, especially in response to economic pressure or public demand.
3. **Inflationary Pressures**: With inflation remaining high, it would not be surprising for the government to change its position if public unrest or economic instability grows.
4. **Reaction to Political Loss**: The loss in local elections could incentivize the ruling party to pursue measures that would quickly boost their popularity, such as a minimum wage increase.

**Adjusted Probability:** 55%

## Critic

I concur.


===========================================================================================================


## Upload

In [19]:
for p in tqdm(predictions.values()):
    p.upload()

 20%|████████████▌                                                  | 1/5 [00:00<00:03,  1.27it/s]

Prediction posted for  26404
Comment posted for  26404


 40%|█████████████████████████▏                                     | 2/5 [00:01<00:02,  1.36it/s]

Prediction posted for  26405
Comment posted for  26405
Prediction posted for  26406


 60%|█████████████████████████████████████▊                         | 3/5 [00:02<00:01,  1.42it/s]

Comment posted for  26406
Prediction posted for  26407


 80%|██████████████████████████████████████████████████▍            | 4/5 [00:02<00:00,  1.42it/s]

Comment posted for  26407


100%|███████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.39it/s]

Prediction posted for  26408
Comment posted for  26408
